# Build a boolean image classifier FAST using any dataset, with a brief explanation of Convolutional Neural Network (CNN) with code

Dataset available at: https://www.kaggle.com/datasets/cashutosh/gender-classification-dataset


In [1]:
!unzip -q "archive.zip"

### Importing required libraries

In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

### Loading data and processing images

In [5]:
BATCH_SIZE = 32

In [48]:
TrainTest_Datagen = ImageDataGenerator(
      rescale=1./255,
      validation_split=0.2 # we will use this as test data
)
Validation_datagen = ImageDataGenerator(
    rescale=1./255,
) 

In [49]:
train_data = TrainTest_Datagen.flow_from_directory(
    './Training',
    target_size=(256,256),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
Test_data = TrainTest_Datagen.flow_from_directory(
    './Training', 
    target_size=(256,256),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'# set as test data
) 
Val_data = Validation_datagen.flow_from_directory(
    './Validation',
    target_size=(256,256),
    batch_size=BATCH_SIZE,
    class_mode='binary',
)


Found 37608 images belonging to 2 classes.
Found 9401 images belonging to 2 classes.
Found 11649 images belonging to 2 classes.


In [50]:
train_data.class_indices

{'female': 0, 'male': 1}

### Modeling and Training

In [51]:
model = Sequential([
    Conv2D(16, (3,3), 1, activation='relu', input_shape = (256,256,3)),
    MaxPooling2D(),

    Conv2D(32, (3,3), 1,  activation='relu'),
    MaxPooling2D(),

    Conv2D(64, (3,3), 1,activation='relu'),
    MaxPooling2D(),

    Flatten(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [52]:
model.compile(optimizer="adam",loss='binary_crossentropy',metrics = ['accuracy'])

In [53]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 127, 127, 16)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 60, 60, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 30, 30, 64)      

In [54]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1)
history = model.fit(
      train_data,
      steps_per_epoch=1175,
      epochs=20,
      validation_data = Val_data,
      validation_steps=364,
      callbacks=[es]
)

Epoch 1/20
1175/1175 [==============================] - 88s 74ms/step - loss: 0.2678 - accuracy: 0.8943 - val_loss: 0.1988 - val_accuracy: 0.9243
Epoch 2/20
1175/1175 [==============================] - 86s 73ms/step - loss: 0.1711 - accuracy: 0.9338 - val_loss: 0.1489 - val_accuracy: 0.9391
Epoch 3/20
1175/1175 [==============================] - 88s 75ms/step - loss: 0.1346 - accuracy: 0.9505 - val_loss: 0.1263 - val_accuracy: 0.9534
Epoch 4/20
1175/1175 [==============================] - 85s 73ms/step - loss: 0.1013 - accuracy: 0.9630 - val_loss: 0.1293 - val_accuracy: 0.9517
Epoch 5/20
1175/1175 [==============================] - 85s 73ms/step - loss: 0.0733 - accuracy: 0.9730 - val_loss: 0.1471 - val_accuracy: 0.9506
Epoch 6/20
1175/1175 [==============================] - 87s 74ms/step - loss: 0.0510 - accuracy: 0.9810 - val_loss: 0.1827 - val_accuracy: 0.9447
Epoch 7/20
1175/1175 [==============================] - 87s 74ms/step - loss: 0.0324 - accuracy: 0.9881 - val_loss: 0.1802 -

### Evaluation

In [63]:
test_loss, test_acc = model.evaluate(Test_data, verbose=0)
print('\nTest accuracy:', np.round(test_acc * 100,3))


Test accuracy: 94.767


### Real time prediction

In [64]:
def RealPredict(img_path):
  img_test = tf.io.read_file(img_path)
  img_test = tf.io.decode_jpeg(img_test , channels = 3)
  img_test = tf.image.resize(img_test , [256 , 256] , method="nearest")
  yhat = model.predict(np.expand_dims(img_test, 0))
  if yhat < 0.5: 
    print(f'Predicted the person a female')
  else:
      print(f'Predicted the person a male')

In [65]:
# {'female': 0, 'male': 1}

In [66]:
RealPredict('Training/female/131422.jpg.jpg')

1/1 [==============================] - 0s 15ms/step
Predicted the person a female


In [67]:
RealPredict('Training/male/090639.jpg.jpg')

1/1 [==============================] - 0s 19ms/step
Predicted the person a male
